In [1]:
import numpy as np
import locale


In [2]:
#for currency
locale.setlocale(locale.LC_ALL, 'en_US.utf8')

#define a seed for the random number generator (to ensure that results are reproducible)
np.random.seed(367)

## defining a class

In [5]:
class Advertising_Campaign():
  
  def __init__(self, campaign_id):
    self.id = campaign_id
    #random conversion rate(1-20%)
    self.conversion_rate = np.random.uniform(0.01, 0.2)
    #random profit
    self.average_profit_per_sale = np.random.uniform(100., 200.)
    #successful or not
    self.sales = 0
    self.no_sales = 0
    #number of times a compaign was tried
  def total_trials(self):
    return self.sales + self.no_sales
  
#profit genereated by a campaign
  def total_profit(self):
    return self.sales * self.average_profit_per_sale

#profit per trial
  def actual_profit_per_trial(self):
    if self.total_trials() > 0:
      return self.total_profit() / self.total_trials()
    else:
      return 0.

#expected profit(though this will never be known in real life)
  def expected_profit_per_trial(self):
    return self.conversion_rate * self.average_profit_per_sale


#try a specific campaign on a customer
def try_campaign(campaign):
  if np.random.random() <= campaign.conversion_rate:
    campaign.sales += 1
  else:
    campaign.no_sales += 1

  

## Environment

In [6]:

number_of_advertising_campaigns = 8

number_of_customers = 10000000

campaigns = []

for c in range(number_of_advertising_campaigns):
  campaigns.append(Advertising_Campaign(c))
  #expected profit per trial(won't be known in real life)
  print('Campaign {0}: Expected profit per trial = {1}'.format(campaigns[c].id, 
                                                               locale.currency(campaigns[c].expected_profit_per_trial(), grouping=True)))
  

Campaign 0: Expected profit per trial = $3.07
Campaign 1: Expected profit per trial = $3.10
Campaign 2: Expected profit per trial = $2.67
Campaign 3: Expected profit per trial = $35.89
Campaign 4: Expected profit per trial = $6.30
Campaign 5: Expected profit per trial = $11.01
Campaign 6: Expected profit per trial = $14.87
Campaign 7: Expected profit per trial = $3.91


## Thompson Sampling

In [7]:
#for every customer
for customer in range(number_of_customers):
  #best beta value and the best campaign
  index_of_campaign_to_try = -1
  best_beta_value = -1

  #for every campaign
  for campaign in campaigns:
    current_campaign_beta_value = np.random.beta(campaign.actual_profit_per_trial() + 1., number_of_advertising_campaigns / 2.)#current beta value
    
    
    if current_campaign_beta_value > best_beta_value:

      best_beta_value = current_campaign_beta_value
   
      index_of_campaign_to_try = campaign.id
  
  #try this campaign on customer
  try_campaign(campaigns[index_of_campaign_to_try])


total_profit_thompson_sampling = 0
total_profit_uniform_sampling = 0

#profit using a naive uniform sampling approach
uniform_customers_per_campaign = number_of_customers / number_of_advertising_campaigns


for campaign in campaigns:
  print('Campaign {0}: Actual profit per trial = {1}, Total trials = {2}'.format(campaign.id, 
                                                                                 locale.currency(campaign.actual_profit_per_trial(), grouping=True), 
                                                                                 campaign.total_trials()))
  total_profit_thompson_sampling += campaign.total_profit()
  total_profit_uniform_sampling += (uniform_customers_per_campaign * campaign.conversion_rate * campaign.average_profit_per_sale)



Campaign 0: Actual profit per trial = $3.05, Total trials = 32685
Campaign 1: Actual profit per trial = $2.92, Total trials = 29835
Campaign 2: Actual profit per trial = $2.87, Total trials = 35490
Campaign 3: Actual profit per trial = $35.84, Total trials = 7836253
Campaign 4: Actual profit per trial = $6.27, Total trials = 160701
Campaign 5: Actual profit per trial = $11.00, Total trials = 620429
Campaign 6: Actual profit per trial = $14.84, Total trials = 1222803
Campaign 7: Actual profit per trial = $3.87, Total trials = 61804


## Results

In [8]:

print('\nThompson Sampling total profit: {0}'.format(locale.currency(total_profit_thompson_sampling, grouping=True)))
print('Uniform Sampling total profit: {0}'.format(locale.currency(total_profit_uniform_sampling, grouping=True)))
print('Thompson Sampling absolute improvement: {0}'.format(locale.currency(total_profit_thompson_sampling - total_profit_uniform_sampling, grouping=True)))
print('Thompson Sampling relative improvement: {0:.2%}'.format((total_profit_thompson_sampling / total_profit_uniform_sampling) - 1.))



Thompson Sampling total profit: $307,331,372.64
Uniform Sampling total profit: $101,023,876.87
Thompson Sampling absolute improvement: $206,307,495.77
Thompson Sampling relative improvement: 204.22%
